In [1]:
import numpy as np 
import matplotlib as plt 
import os 
import time
import cv2 
import pandas as pd
import tensorflowjs as tfjs
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers,models,activations
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Input,LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

C:\Users\andre\Anaconda3\envs\tf25\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\andre\Anaconda3\envs\tf25\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\andre\Anaconda3\envs\tf25\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load and Preprocess Data

In [6]:
DATA_PATH = os.path.DATA_PATH = os.path.join(os.getcwd(),"Fingerpose_data")
gestures = os.listdir(DATA_PATH)
curl_map = {'No Curl':0,'Half Curl':1,'Full Curl':2}
direction_map = {'Vertical Up':0,'Vertical Down':1,'Horizontal Left':2,'Horizontal Right':3,'Diagonal Up Right':4,
                 'Diagonal Up Left':5,'Diagonal Down Right':6,'Diagonal Down Left':7}
label_map = {label:num for num, label in enumerate(gestures)}

In [7]:
#read data from CSV file and save to a dataset 
def load_csv(data,gesture,dataset,labels):
    data = data.reset_index()
    frame,datapoints = [],[]
    for index, row in data.iterrows():
        frame.append(curl_map[row['Curl']])
        frame.append(direction_map[row['Direction']])
        if (index+1)%5 == 0:
            datapoints.append(frame)
            frame = []
        if (index+1)%150 == 0:
            dataset.append(datapoints)
            labels.append(label_map[gesture])
            datapoints=[]

In [8]:
dataset,labels = [],[]
for gesture in gestures:
    files = os.listdir(os.path.join(DATA_PATH,gesture))
    for file in files:
        df = pd.read_csv(os.path.join(DATA_PATH,gesture,file))
        load_csv(df,gesture,dataset,labels)
        
            

In [9]:
X = np.array(dataset)
y = to_categorical(labels).astype(int)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

(60, 30, 10)
(60, 3)


## Train LSTM Model 

In [10]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
inputs = Input((30,10))
lstm = LSTM(64,return_sequences=True, activation = 'relu',input_shape=(30,126))(inputs)
lstm2 = LSTM(128, activation = 'relu')(lstm)
de = Dense(128, activation='relu')(lstm2)
de2 = Dense(64, activation='relu')(de)
outputs = Dense(len(gestures),activation='softmax')(de2)
model = models.Model(inputs,outputs)
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 10)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 30, 64)            19200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 142,979
Trainable params: 142,979
Non-trainable params: 0
_______________________________________________________

In [12]:
model.fit(X_train, y_train, epochs=250, callbacks=[tb_callback])

Epoch 1/250
2/2 [==============================] - 4s 886ms/step - loss: 1.1294 - categorical_accuracy: 0.4074
Epoch 2/250
2/2 [==============================] - 1s 253ms/step - loss: 0.8726 - categorical_accuracy: 0.6667
Epoch 3/250
2/2 [==============================] - 1s 288ms/step - loss: 0.6454 - categorical_accuracy: 0.7778
Epoch 4/250
2/2 [==============================] - 1s 286ms/step - loss: 0.4291 - categorical_accuracy: 0.9074
Epoch 5/250
2/2 [==============================] - 1s 284ms/step - loss: 0.5699 - categorical_accuracy: 0.8333
Epoch 6/250
2/2 [==============================] - 1s 324ms/step - loss: 0.5164 - categorical_accuracy: 0.7778
Epoch 7/250
2/2 [==============================] - 1s 268ms/step - loss: 0.3900 - categorical_accuracy: 0.8704
Epoch 8/250
2/2 [==============================] - 1s 253ms/step - loss: 0.3531 - categorical_accuracy: 0.8519
Epoch 9/250
2/2 [==============================] - 0s 246ms/step - loss: 0.2204 - categorical_accuracy: 0.9259
E

2/2 [==============================] - 1s 312ms/step - loss: 1.2454e-05 - categorical_accuracy: 1.0000
Epoch 73/250
2/2 [==============================] - 1s 270ms/step - loss: 1.1807e-05 - categorical_accuracy: 1.0000
Epoch 74/250
2/2 [==============================] - 1s 238ms/step - loss: 1.1194e-05 - categorical_accuracy: 1.0000
Epoch 75/250
2/2 [==============================] - 1s 248ms/step - loss: 1.0739e-05 - categorical_accuracy: 1.0000
Epoch 76/250
2/2 [==============================] - 1s 247ms/step - loss: 1.0218e-05 - categorical_accuracy: 1.0000
Epoch 77/250
2/2 [==============================] - 1s 304ms/step - loss: 9.6885e-06 - categorical_accuracy: 1.0000
Epoch 78/250
2/2 [==============================] - 1s 270ms/step - loss: 9.3132e-06 - categorical_accuracy: 1.0000
Epoch 79/250
2/2 [==============================] - 1s 289ms/step - loss: 8.9534e-06 - categorical_accuracy: 1.0000
Epoch 80/250
2/2 [==============================] - 1s 260ms/step - loss: 8.5539e-06 

2/2 [==============================] - 1s 285ms/step - loss: 2.2186e-06 - categorical_accuracy: 1.0000
Epoch 143/250
2/2 [==============================] - 1s 249ms/step - loss: 2.1877e-06 - categorical_accuracy: 1.0000
Epoch 144/250
2/2 [==============================] - 1s 320ms/step - loss: 2.1546e-06 - categorical_accuracy: 1.0000
Epoch 145/250
2/2 [==============================] - 1s 295ms/step - loss: 2.1259e-06 - categorical_accuracy: 1.0000
Epoch 146/250
2/2 [==============================] - 1s 305ms/step - loss: 2.0972e-06 - categorical_accuracy: 1.0000
Epoch 147/250
2/2 [==============================] - 1s 308ms/step - loss: 2.0729e-06 - categorical_accuracy: 1.0000
Epoch 148/250
2/2 [==============================] - 1s 301ms/step - loss: 2.0420e-06 - categorical_accuracy: 1.0000
Epoch 149/250
2/2 [==============================] - 1s 291ms/step - loss: 2.0044e-06 - categorical_accuracy: 1.0000
Epoch 150/250
2/2 [==============================] - 1s 297ms/step - loss: 1.9

2/2 [==============================] - 1s 242ms/step - loss: 8.3667e-07 - categorical_accuracy: 1.0000
Epoch 213/250
2/2 [==============================] - 1s 266ms/step - loss: 8.2784e-07 - categorical_accuracy: 1.0000
Epoch 214/250
2/2 [==============================] - 1s 320ms/step - loss: 8.2121e-07 - categorical_accuracy: 1.0000
Epoch 215/250
2/2 [==============================] - 0s 242ms/step - loss: 8.1238e-07 - categorical_accuracy: 1.0000
Epoch 216/250
2/2 [==============================] - 1s 259ms/step - loss: 8.0135e-07 - categorical_accuracy: 1.0000
Epoch 217/250
2/2 [==============================] - 1s 288ms/step - loss: 7.9472e-07 - categorical_accuracy: 1.0000
Epoch 218/250
2/2 [==============================] - 1s 341ms/step - loss: 7.8810e-07 - categorical_accuracy: 1.0000
Epoch 219/250
2/2 [==============================] - 1s 302ms/step - loss: 7.8148e-07 - categorical_accuracy: 1.0000
Epoch 220/250
2/2 [==============================] - 1s 313ms/step - loss: 7.7

In [26]:
#convert and save file in tfjs format 
tfjs.converters.save_keras_model(model,os.getcwd())

In [ ]:
#save model in python format
model.save('gestures.h5')